In [5]:
import requests
import pandas as pd
import json
import mysql.connector
from datetime import datetime, timedelta, timezone
import time
import nvda_config
import twelvedata as td
from twelvedata import TDClient


In [6]:
# set up login
Cpassword = nvda_config.mysql_password
td = TDClient(apikey=nvda_config.twelve_api_nvda) 
base_url = 'https://api.twelvedata.com'


In [7]:
# Estab;ish MySQL object
mydb = mysql.connector.connect(
    host="localhost",   
    user="root",         
    password=Cpassword,  
    database="nvdav1"   
)

mycursor = mydb.cursor()




In [8]:


# Twelve Data API parameters
symbol = "NVDA"
interval = "1min"
outputsize = 5000 


while True:
    # Fetch time series data and indicators
    price_pdf = td.price(
        symbol=symbol
    )
    # Convert to pandas Dataframes
    price_json = price_pdf.as_json()
    price_df = pd.DataFrame(price_json, index=["price"])

    # Prepare for database insertion
    time_series_data = []

    # Process and align data
    for index, row in price_df.iterrows():
        # Generate the current datetime string
        datetime_str = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

        time_series_data.append(
            (
                symbol,
                datetime_str,
                row["price"]
            )
        )

    # Insert tick data into MySQL
    sql_time_series = """
    INSERT INTO nvda_new (symbol, datetime, price)
    VALUES (%s, %s, %s)
    """
    mycursor.executemany(sql_time_series, time_series_data)
    mydb.commit()

    # Sleep for a short period before fetching the next data
    time.sleep(60)  # Adjust the sleep time as needed


: 